# Fazendo análise dos dados da covid_2019

### Digital inovation one(Dio.me)

In [ ]:
# Importando os pacotes para a análise do dataset

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Carregando o dataFrame
# Transformando as colunas de data para o valor date

df = pd.read_csv('covid_19_data.csv', parse_dates=['ObservationDate', 'Last Update'])
df.head(5)

In [ ]:
# Analisando os tipos de dados do datFrame

df.dtypes

Nomes de colunas não devem ter letras maiúsculas e nem caracteres especiais.

Será implementado uma função para fazer a limpeza dos nomes dessas colunas

In [ ]:
import re # para expressões regulares
# função para corrigir os nomes da colunas

def corrige_coluna(col_name):
     return re.sub(r"[/| ]", "", col_name).lower()

In [ ]:
# Corrigindo o nome das colunas
# transformando para minúsculo o nome de todas as colunas, e eliminando os espaços em branco e os carcteres especiais

df.columns = [corrige_coluna(col) for col in df.columns]
df.head()

## Selecionando apenas os dados do brasil para fazer as análises

In [ ]:
# verificando os valores únicos do dataFrame

df.countryregion.unique()

In [ ]:
# Filtrando os dados usando o método loc
# filtrando apenas por brasil(Brazil)

df.loc[df.countryregion == 'Brazil']

In [ ]:
# Criando um dataFrame somente para brasil
# com casos confirmados maior que zero

brasil = df.loc[
    (df.countryregion == 'Brazil') &
    (df.confirmed > 0)
]

## Analisando os casos confirmados

In [ ]:
# Criando um gráfico da evolução de casos confirmados

px.line(brasil, 'observationdate', 'confirmed', title='Casos confirmados no Brasil')

## Usando Lambda para identificar novos casos por dia

In [ ]:
# Usando programação funcional para novos casos a cada dia
# criando uma nova coluna

brasil['novoscasos'] = list(map(
    lambda x: 0 if x==0 else brasil['confirmed'].iloc[x] - brasil['confirmed'].iloc[x-1],
    np.arange(brasil.shape[0])
))

In [ ]:
# Visualisando

px.line(brasil, x='observationdate',y='novoscasos',title='Novos casoso por dia')

# Mortes

In [ ]:
# Organizando a camada de dados
# número de mortes por covid a cada dia no brasil

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=brasil.observationdate, y=brasil.deaths, name='Mortes',
              mode='lines+markers', line={'color':'red'})
)

# Organizando o layout

fig.update_layout(title='Mortes por covid-19 no Brasil')
fig.show()

# Taxa de média de crescimento

In [ ]:
# Criando uma funão para verificar a taxa média de crescimento diário

def taxa_crescimento(data, variable, data_inicio=None, data_fim=None):
    # se data inicio for None, define como a primeira data disponivel
    if data_inicio == None:
        data_inicio = data.observationdate.loc[data[variable] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)

    if data_fim == None:
        data_fim = data.observationdate.iloc[-1]
    else:
        data_fim = pd.to_datetime(data_fim)

    # Definir os valores de presente e passado
    passado = data.loc[data.observationdate == data_inicio, variable].values[0]
    presente = data.loc[data.observationdate == data_fim, variable].values[0]

    # Define o número de pontos no tempo que vamos avaliar
    n = (data_fim - data_inicio).days

    # Calcular a taxa
    taxa = (presente/passado)**(1/n) - 1

    return taxa*100

In [ ]:
# Taxa do crescimento médio em todo o brasil

taxa_crescimento(brasil, 'confirmed')

# Taxa de crescimento diário

In [ ]:
# Criando uma função para taxa de crescimento diário

def taxa_crescimento_diario(data, variable, data_inicio=None):
    # se data inicio for None, define como a primeira data disponivel
    if data_inicio == None:
        data_inicio = data.observationdate.loc[data[variable] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)

    data_fim = data.observationdate.max()

    # Define o número de pontos no tempo que vamos avaliar
    n = (data_fim-data_inicio).days

    # Taxa calculada de um dia para o outro
    taxas = list(map(
        lambda x: (data[variable].iloc[x] - data[variable].iloc[x-1]) / data[variable].iloc[x-1],
        range(1, n+1)
    ))
    return np.array(taxas) * 100

In [ ]:
tx_dia = taxa_crescimento_diario(brasil, 'confirmed')

In [ ]:
tx_dia

In [ ]:
# Plotando no gráfico

primeiro_dia = brasil.observationdate.loc[brasil.confirmed > 0].min()

px.line(x=pd.date_range(primeiro_dia, brasil.observationdate.max())[1:], y=tx_dia, title='Taxa de crescimento de casos confirmados no Brasil')

# Realisando cálculos de predição

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
confirmados = brasil.confirmed
confirmados.index = brasil.observationdate
confirmados

In [ ]:
res = seasonal_decompose(confirmados)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10,8))

ax1.plot(res.observed)
ax2.plot(res.trend)
ax3.plot(res.seasonal)
ax4.plot(confirmados.index, res.resid)
ax4.axhline(0, linestyle='dashed',c='black')
plt.show()

# Fazendo a modelagem com ARIMA

In [ ]:
from pmdarima.arima import auto_arima

modelo = auto_arima(confirmados)

In [ ]:
fig = go.Figure(go.Scatter(
    x= confirmados.index, y=confirmados, name='Observados'
))
fig.add_trace(go.Scatter(
    x=confirmados.index, y=modelo.predict_in_sample(), name='Preditos'
))
fig.add_trace(go.Scatter(
    x=pd.date_range('2020-05-20','2020-06-20'), y=modelo.predct(31), name='Forecast'
))
fig.update_layout(title='Previsão de casos confirmados no Brasil par os próximos 30 dias')
fig.show()

# Modelo de crescimento

In [ ]:
# Preprocessamentos

from asyncio import futures


train = confirmados.reset_index()[:5]
test = confirmados.reset_index()[-5:]

# Renomeando colunas
train.rename(columns={'observationdate':'ds', 'confirmed':'y'}, inplace=True)
test.rename(columns={'observationdate':'ds', 'confirmed':'y'}, inplace=True)

# Defenir o modelo de crescimento
profeta = Prophet(grow='logistic',changepoints=['2020-03-21', '2020-03-30','2020-04-25',
                                                '2020-05-03','2020-05-10'])

pop = 211463256
train['cap'] = pop

# Treina o modelo
profeta.fit(train)

# Construir previsões para o futuro
future_dates = profeta.make_future_dataframe(periods=200)
future_dates['cap'] = pop
forecast = profeta.predict(future_dates)

In [ ]:
# Gráfico para mostrar a predição

fig = go.Figure()

fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat, name='Predição'))
fig.add_trace(go.Scatter(x=test.index, y=test, name='Observados - Teste'))
fig.add_trace(go.Scatter(x=train.ds, y=train.y, name='Observados - Treino'))
fig.update_layout(title='Predições de casos confirmados no Brasil')
fig.show()